In [1]:
# modules required for handling dataframes
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from decimal import Decimal

In [2]:
# put in all input parameters. Here I am showing the code for one sample as an example.
# to generate the final_df for other samples, simply change the basedir and barcode, as all file names just has this two difference between each two samples.
# please note that there are other places of this script that require understanding and hard coding skills which I also commented below.

sourcedir = '/home/yiheng/MinION_data' # the directory where all the documents of each sequencing run are stored.
barcode = '05' # the barcode for each sample. Barcode05 is PD sample, 06 is PB sample.
basedir = os.path.join(sourcedir, 'barcode%s' % barcode)
db = "refseq_fungi_updated" # database used
genera_in_mock = ['Aspergillus','Blastobotrys','Candida','Diutina', 'Nakaseomyces', 'Clavispora','Cryptococcus','Cyberlindnera',
'Debaryomyces','Geotrichum','Kluyveromyces','Kodamaea','Lomentospora','Magnusiomyces','Meyerozyma','Pichia',
'Rhodotorula','Scedosporium','Trichophyton', 'Trichosporon', 'Wickerhamomyces','Yarrowia','Zygoascus', 'Purpureocillium']

In [3]:
blastdf_dir = os.path.join(basedir, 'barcode%s.%sdb_blast.tab' % (barcode,db)) # the directory for .blast_output file
blast_df = pd.read_csv(blastdf_dir, index_col=0, sep='\t')

In [4]:
# this is for the total # of bases, which is total number of reads passed QC (7)
seq_sum_dir = os.path.join(basedir, 'sequencing_summary_barcode%s.txt' % barcode) # the directory of sequencing summary file for each run
seq_sum_df = pd.read_csv(seq_sum_dir, sep='\t')
seq_sum_df_pass = seq_sum_df[seq_sum_df.passes_filtering==True]
seq_sum_df_pass.sequence_length_template.sum()

2223839802

In [5]:
seq_sum_df_pass.columns

Index(['filename', 'read_id', 'run_id', 'batch_id', 'channel', 'mux',
       'start_time', 'duration', 'num_events', 'passes_filtering',
       'template_start', 'num_events_template', 'template_duration',
       'sequence_length_template', 'mean_qscore_template',
       'strand_score_template', 'median_template', 'mad_template',
       'scaling_median_template', 'scaling_mad_template'],
      dtype='object')

In [6]:
seq_sum_df_pass_drop = seq_sum_df_pass.drop(columns=['batch_id', 'filename', 'run_id', 'channel', 'mux', 'start_time',
       'duration', 'num_events', 'passes_filtering', 'template_start','num_events_template', 'template_duration', 
        'strand_score_template', 'median_template', 'mad_template', 'scaling_median_template', 'scaling_mad_template'])
subset_seq_sum_df_unclassified = seq_sum_df_pass_drop[~seq_sum_df_pass_drop.read_id.isin(blast_df.qseqid_blast)]
subset_seq_sum_df_pass = seq_sum_df_pass_drop[seq_sum_df_pass_drop.read_id.isin(blast_df.qseqid_blast)]

In [7]:
total_blast_df = pd.merge(blast_df, seq_sum_df_pass_drop,how="outer", left_on='qseqid_blast', right_on='read_id')
total_blast_df['superkingdom_blast'] = 'Eukaryota'
total_blast_df['sequence_length_template_blast'] = total_blast_df['sequence_length_template']
total_blast_fillna = total_blast_df.fillna(value='unclassified')
total_blast_fillna[total_blast_fillna.pident_blast != 'unclassified'].sequence_length_template.sum()

1618364905

In [8]:
final_df = pd.merge(blast_df, subset_seq_sum_df_pass,how="outer", left_on='qseqid_blast', right_on='read_id')

In [9]:
def generate_taxonomy_pivot_sum_blast(tax_df, rank, bcs, num):
    """From tax_df, generate a pivot table listing num rank counts, sorted by bcs"""
    pivot_table = tax_df.pivot_table(values='sequence_length_template_blast', 
                                            index=rank, 
                                            columns='superkingdom_blast', 
                                            aggfunc='sum', 
                                            fill_value=0)
    pivot_table.columns.name = None
    pivot_table = pivot_table.sort_values(bcs, axis=0, ascending=False).head(n=num)
    return pivot_table

In [10]:
# Alright, here is the math. 
# Actually they can be all merged into one function but no harm to keep it like that as it's clearer.
# Here the pmatch is the query coverage.
def calculate_precision_pmatch(blast_df, pmatch):
    subset_blast_df = blast_df[blast_df.pmatch_blast >= float(pmatch)]
    fungidb_blast_pivot = generate_taxonomy_pivot_sum_blast(subset_blast_df, 'genus_blast', 'Eukaryota', 2000) 
    subset_fungidb_blast_pivot = fungidb_blast_pivot[fungidb_blast_pivot.index.isin(genera_in_mock)]
    precision = float((subset_fungidb_blast_pivot.sum().sum())/fungidb_blast_pivot['Eukaryota'].sum()*100)
    return precision

def calculate_precision_pident(blast_df, pident):
    subset_blast_df = blast_df[blast_df.pident_blast >= float(pident)]
    fungidb_blast_pivot = generate_taxonomy_pivot_sum_blast(subset_blast_df, 'genus_blast', 'Eukaryota', 2000) 
    subset_fungidb_blast_pivot = fungidb_blast_pivot[fungidb_blast_pivot.index.isin(genera_in_mock)]
    precision = float((subset_fungidb_blast_pivot.sum().sum())/fungidb_blast_pivot['Eukaryota'].sum()*100)
    return precision

def calculate_precision_length(blast_df, length):
    subset_blast_df = blast_df[blast_df.sequence_length_template_blast >= float(length)]
    fungidb_blast_pivot = generate_taxonomy_pivot_sum_blast(subset_blast_df, 'genus_blast', 'Eukaryota', 2000) 
    subset_fungidb_blast_pivot = fungidb_blast_pivot[fungidb_blast_pivot.index.isin(genera_in_mock)]
    precision = float((subset_fungidb_blast_pivot.sum().sum())/fungidb_blast_pivot['Eukaryota'].sum()*100)
    return precision

def calculate_precision_evalue(blast_df, evalue):
    subset_blast_df = blast_df[blast_df.log_evalue_blast >= int(evalue)]
    fungidb_blast_pivot = generate_taxonomy_pivot_sum_blast(subset_blast_df, 'genus_blast', 'Eukaryota', 2000) 
    subset_fungidb_blast_pivot = fungidb_blast_pivot[fungidb_blast_pivot.index.isin(genera_in_mock)]
    precision = float((subset_fungidb_blast_pivot.sum().sum())/fungidb_blast_pivot['Eukaryota'].sum()*100)
    return precision

def calculate_precision_qscore(blast_df, qscore):
    subset_blast_df = blast_df[blast_df.mean_qscore_template >= int(qscore)]
    fungidb_blast_pivot = generate_taxonomy_pivot_sum_blast(subset_blast_df, 'genus_blast', 'Eukaryota', 2000) 
    subset_fungidb_blast_pivot = fungidb_blast_pivot[fungidb_blast_pivot.index.isin(genera_in_mock)]
    precision = float(subset_fungidb_blast_pivot.sum()/fungidb_blast_pivot.sum()*100)
    return precision

def calculate_completeness_pmatch(blast_df, pmatch):
    subset_blast_df = blast_df[blast_df.pmatch_blast >= float(pmatch)]
    fungidb_blast_pivot = generate_taxonomy_pivot_sum_blast(subset_blast_df, 'genus_blast', 'Eukaryota', 2000) 
    subset_fungidb_blast_pivot = fungidb_blast_pivot[fungidb_blast_pivot.index.isin(genera_in_mock)]
# 24 is the number of genera that in the community, including the contaminative species and those alternative names
    completeness = len(subset_fungidb_blast_pivot)/24*100
    return completeness

def calculate_completeness_pident(blast_df, pident):
    subset_blast_df = blast_df[blast_df.pident_blast >= float(pident)]
    fungidb_blast_pivot = generate_taxonomy_pivot_sum_blast(subset_blast_df, 'genus_blast', 'Eukaryota', 2000) 
    subset_fungidb_blast_pivot = fungidb_blast_pivot[fungidb_blast_pivot.index.isin(genera_in_mock)]
    completeness = len(subset_fungidb_blast_pivot)/24*100
    return completeness

def calculate_completeness_length(blast_df, length):
    subset_blast_df = blast_df[blast_df.sequence_length_template_blast >= float(length)]
    fungidb_blast_pivot = generate_taxonomy_pivot_sum_blast(subset_blast_df, 'genus_blast', 'Eukaryota', 2000) 
    subset_fungidb_blast_pivot = fungidb_blast_pivot[fungidb_blast_pivot.index.isin(genera_in_mock)]
    completeness = len(subset_fungidb_blast_pivot)/24*100
    return completeness

def calculate_completeness_evalue(blast_df, evalue):
    subset_blast_df = blast_df[blast_df.log_evalue_blast >= int(evalue)]
    fungidb_blast_pivot = generate_taxonomy_pivot_sum_blast(subset_blast_df, 'genus_blast', 'Eukaryota', 2000) 
    subset_fungidb_blast_pivot = fungidb_blast_pivot[fungidb_blast_pivot.index.isin(genera_in_mock)]
    completeness = len(subset_fungidb_blast_pivot)/24*100
    return completeness

def calculate_completeness_qscore(blast_df, qscore):
    subset_blast_df = blast_df[blast_df.mean_qscore_template >= int(qscore)]
    fungidb_blast_pivot = generate_taxonomy_pivot_sum_blast(subset_blast_df, 'genus_blast', 'Eukaryota', 2000) 
    subset_fungidb_blast_pivot = fungidb_blast_pivot[fungidb_blast_pivot.index.isin(genera_in_mock)]
    # 24 is the number of genera that in the community, including the contaminative species and those alternative names
    completeness = len(subset_fungidb_blast_pivot)/24*100
    return completeness

In [11]:
# Please not that there is a difference in the denominator of calculating remaining rate.
# This is to reflect the 'actual' remaining rate before we run the alignement.
# For all other alignment parameters, we cannot do it because they are dependent on the alignment result.
# For Illumina data, it was not much unaligned contigs so I just used the total number of aligned contigs for the calculation.
def calculate_remaining_length(blast_df, length):
    if barcode == '06':
        subset_blast_df = blast_df[blast_df.sequence_length_template_blast >= float(length)]
        fungidb_blast_pivot = generate_taxonomy_pivot_sum_blast(subset_blast_df, 'genus_blast', 'Eukaryota', 20000) 
        remaining_rate = float(fungidb_blast_pivot.sum().sum()/4052192342*100)
        return remaining_rate
    elif barcode == '05':
        subset_blast_df = blast_df[blast_df.sequence_length_template_blast >= float(length)]
        fungidb_blast_pivot = generate_taxonomy_pivot_sum_blast(subset_blast_df, 'genus_blast', 'Eukaryota', 20000) 
        remaining_rate = float(fungidb_blast_pivot.sum().sum()/2223839802*100)
        return remaining_rate

def calculate_remaining_evalue(blast_df, evalue):
    if barcode == '06':
        subset_blast_df = blast_df[blast_df.log_evalue_blast >= int(evalue)]
        fungidb_blast_pivot = generate_taxonomy_pivot_sum_blast(subset_blast_df, 'genus_blast', 'Eukaryota', 20000) 
        remaining_rate = float(fungidb_blast_pivot.sum()/2905573749*100)
        return remaining_rate
    elif barcode == '05':
        subset_blast_df = blast_df[blast_df.log_evalue_blast >= int(evalue)]
        fungidb_blast_pivot = generate_taxonomy_pivot_sum_blast(subset_blast_df, 'genus_blast', 'Eukaryota', 20000) 
        remaining_rate = float(fungidb_blast_pivot.sum()/1618364905*100)
        return remaining_rate

def calculate_remaining_pmatch(blast_df, pmatch):
    if barcode == '06':
        subset_blast_df = blast_df[blast_df.pmatch_blast >= int(pmatch)]
        fungidb_blast_pivot = generate_taxonomy_pivot_sum_blast(subset_blast_df, 'genus_blast', 'Eukaryota', 20000) 
        remaining_rate = float(fungidb_blast_pivot.sum()/2905573749*100)
        return remaining_rate
    elif barcode == '05':
        subset_blast_df = blast_df[blast_df.pmatch_blast >= int(pmatch)]
        fungidb_blast_pivot = generate_taxonomy_pivot_sum_blast(subset_blast_df, 'genus_blast', 'Eukaryota', 20000) 
        remaining_rate = float(fungidb_blast_pivot.sum()/1618364905*100)
        return remaining_rate
    
def calculate_remaining_pident(blast_df, pident):
    if barcode == '06':
        subset_blast_df = blast_df[blast_df.pident_blast >= int(pident)]
        fungidb_blast_pivot = generate_taxonomy_pivot_sum_blast(subset_blast_df, 'genus_blast', 'Eukaryota', 20000) 
        remaining_rate = float(fungidb_blast_pivot.sum()/2905573749*100)
        return remaining_rate
    elif barcode == '05':
        subset_blast_df = blast_df[blast_df.pident_blast >= int(pident)]
        fungidb_blast_pivot = generate_taxonomy_pivot_sum_blast(subset_blast_df, 'genus_blast', 'Eukaryota', 20000) 
        remaining_rate = float(fungidb_blast_pivot.sum()/3363783952*100)
        return remaining_rate

def calculate_remaining_qscore(blast_df, qscore):
    if barcode == '06':
        subset_blast_df = blast_df[blast_df.mean_qscore_template >= float(qscore)]
        fungidb_blast_pivot = generate_taxonomy_pivot_sum_blast(subset_blast_df, 'genus_blast', 'Eukaryota', 2000) 
        remaining_rate = float(fungidb_blast_pivot.sum()/4052192342*100)
        return remaining_rate
    elif barcode == '05':
        subset_blast_df = blast_df[blast_df.mean_qscore_template >= float(qscore)]
        fungidb_blast_pivot = generate_taxonomy_pivot_sum_blast(subset_blast_df, 'genus_blast', 'Eukaryota', 2000) 
        remaining_rate = float(fungidb_blast_pivot.sum()/2223839802*100)
        return remaining_rate    


In [12]:
qscore_precision_df = pd.DataFrame()
# Here the range may need to be hard coded according to the qscore range.
qscore_precision_df['qscore'] = np.arange(7, 20, 0.1)
qscore_precision_df['precision'] = np.nan

for qscore in qscore_precision_df['qscore']:
    qscore_precision_df.iloc[int(qscore_precision_df[qscore_precision_df['qscore']==qscore].index[0]),
                             qscore_precision_df.columns.get_loc('precision')] = calculate_precision_qscore(final_df, qscore)

qscore_precision_df.to_csv(os.path.join(basedir, 'barcode%s.%sdb_qscore_precision.tab' % (barcode, db)), sep='\t')

In [13]:
qscore_completeness_df = pd.DataFrame()
qscore_completeness_df['qscore'] = np.arange(7, 20.6, 0.1)
qscore_completeness_df['completeness'] = np.nan

for qscore in qscore_completeness_df['qscore']:
    qscore_completeness_df.iloc[int(qscore_completeness_df[qscore_completeness_df['qscore']==qscore].index[0]),
                          qscore_completeness_df.columns.get_loc('completeness')] = calculate_completeness_qscore(final_df, qscore)

qscore_completeness_df.to_csv(os.path.join(basedir, 'barcode%s.%sdb_qscore_completeness.tab' % (barcode, db)), sep='\t')

In [14]:
# log_evalue normaly ends at 414 or 415, the computer will not recognize if higher than that.
evalue_x_precision = pd.DataFrame()
evalue_x_precision['evalue'] = range(0, 414)
evalue_x_precision['precision_rate'] = np.nan

for evalue in range(0, 414):
    evalue_x_precision.iloc[evalue, evalue_x_precision.columns.get_loc('precision_rate')] = calculate_precision_evalue(final_df, evalue)

evalue_x_precision.to_csv(os.path.join(basedir, 'barcode%s.%sdb_evalue_precision.tab' % (barcode, db)), sep='\t')

In [15]:
evalue_x_completeness = pd.DataFrame()
evalue_x_completeness['evalue'] = range(0, 414)
evalue_x_completeness['completeness'] = np.nan

for evalue in range(0, 414):
    evalue_x_completeness.iloc[evalue, evalue_x_completeness.columns.get_loc('completeness')] = calculate_completeness_evalue(final_df, evalue)

evalue_x_completeness.to_csv(os.path.join(basedir, 'barcode%s.%sdb_evalue_completeness.tab' % (barcode, db)), sep='\t')

In [16]:
# This is to help decide the range of the read length cut-offs.
total_blast_fillna.sequence_length_template.sort_values(ascending=False).head()

765490     69313
312573     67024
1234739    60274
373434     51032
463092     50829
Name: sequence_length_template, dtype: int64

In [17]:
length_x_precision = pd.DataFrame()
length_x_precision['length'] = np.arange(0,20000,50)
length_x_precision['precision_rate'] = np.nan

for length in length_x_precision['length']:
    length_x_precision.iloc[int(length_x_precision[length_x_precision['length']==length].index[0]),
                          length_x_precision.columns.get_loc('precision_rate')] = calculate_precision_length(final_df, length)

length_x_precision.to_csv(os.path.join(basedir, 'barcode%s.%sdb_length_precision.tab' % (barcode, db)), sep='\t')

In [18]:
length_x_completeness = pd.DataFrame()
length_x_completeness['length'] = np.arange(0,20000,50)
length_x_completeness['completeness'] = np.nan

for length in length_x_completeness['length']:
    length_x_completeness.iloc[int(length_x_completeness[length_x_completeness['length']==length].index[0]), 
                               length_x_completeness.columns.get_loc('completeness')] = calculate_completeness_length(final_df, length)
    
length_x_completeness.to_csv(os.path.join(basedir, 'barcode%s.%sdb_length_completeness.tab' % (barcode, db)), sep='\t')

In [19]:
pident_x_precision = pd.DataFrame()
pident_x_precision['pident'] = range(0, 101)
pident_x_precision['precision_rate'] = np.nan

for pident in pident_x_precision['pident']:
    pident_x_precision.iloc[pident_x_precision[pident_x_precision.pident==pident].index, 
                            pident_x_precision.columns.get_loc('precision_rate')] = calculate_precision_pident(final_df, pident)

pident_x_precision.to_csv(os.path.join(basedir, 'barcode%s.%sdb_pident_precision.tab' % (barcode, db)), sep='\t')

In [20]:
pident_x_completeness = pd.DataFrame()
pident_x_completeness['pident'] = range(0, 101)
pident_x_completeness['completeness'] = np.nan

for pident in range(0,101):
    pident_x_completeness.iloc[pident, pident_x_completeness.columns.get_loc('completeness')] = calculate_completeness_pident(final_df, pident)
    
pident_x_completeness.to_csv(os.path.join(basedir, 'barcode%s.%sdb_pident_completeness.tab' % (barcode, db)), sep='\t')

In [21]:
# this is to see what range of pident in our dataframe
final_df.pmatch_blast.sort_values(ascending=False).head()

592058    98.465420
763171    97.902012
453485    97.894057
399225    97.890576
263340    97.764545
Name: pmatch_blast, dtype: float64

In [22]:
pmatch_X_precision = pd.DataFrame()
pmatch_X_precision['pmatch'] = range(0, 98)
pmatch_X_precision['precision_rate'] = np.nan

for pmatch in pmatch_X_precision['pmatch']:
    pmatch_X_precision.iloc[pmatch_X_precision[pmatch_X_precision.pmatch==pmatch].index, 
                            pmatch_X_precision.columns.get_loc('precision_rate')] = calculate_precision_pmatch(final_df, pmatch)

pmatch_X_precision.to_csv(os.path.join(basedir, 'barcode%s.%sdb_pmatch_precision.tab' % (barcode, db)), sep='\t')

In [23]:
pmatch_X_completeness = pd.DataFrame()
pmatch_X_completeness['pmatch'] = range(0, 98)
pmatch_X_completeness['completeness'] = np.nan

for pmatch in range(0,98):
    pmatch_X_completeness.iloc[pmatch, pmatch_X_completeness.columns.get_loc('completeness')] = calculate_completeness_pmatch(final_df, pmatch)

pmatch_X_completeness.to_csv(os.path.join(basedir, 'barcode%s.%sdb_pmatch_completeness.tab' % (barcode, db)), sep='\t')

In [24]:
length_x_remaining = pd.DataFrame()
# This number of 20000 is from the total sum of the sequence length.
length_x_remaining['length'] = np.arange(0, 20000, 50)
length_x_remaining['remaining_rate'] = np.nan

for length in length_x_remaining['length']:
    length_x_remaining.iloc[int(length_x_remaining[length_x_remaining['length']==length].index[0]),
                            length_x_remaining.columns.get_loc('remaining_rate')] = calculate_remaining_length(total_blast_fillna, length)

length_x_remaining.to_csv(os.path.join(basedir, 'barcode%s.%sdb_length_remaining.tab' % (barcode, db)), sep='\t')    

In [25]:
qscore_x_remaining = pd.DataFrame()
qscore_x_remaining['qscore'] = np.arange(7, 20.1, 0.1)
qscore_x_remaining['remaining_rate'] = np.nan

for qscore in qscore_x_remaining['qscore']:
    qscore_x_remaining.iloc[qscore_x_remaining[qscore_x_remaining.qscore==qscore].index,
                            qscore_x_remaining.columns.get_loc('remaining_rate')] = calculate_remaining_qscore(total_blast_fillna, qscore)

qscore_x_remaining.to_csv(os.path.join(basedir, 'barcode%s.%sdb_qscore_remaining.tab' % (barcode, db)), sep='\t')

In [26]:
pident_x_remaining = pd.DataFrame()
pident_x_remaining['pident'] = range(0, 100)
pident_x_remaining['remaining_rate'] = np.nan

for y in pident_x_remaining['pident']:
    pident_x_remaining.iloc[pident_x_remaining[pident_x_remaining.pident==y].index,
                            pident_x_remaining.columns.get_loc('remaining_rate')] = calculate_remaining_pident(final_df, y)

pident_x_remaining.to_csv(os.path.join(basedir, 'barcode%s.%sdb_pident_remaining.tab' % (barcode, db)), sep='\t')

In [27]:
pmatch_x_remaining = pd.DataFrame()
pmatch_x_remaining['pmatch'] = range(0, 98)
pmatch_x_remaining['remaining_rate'] = np.nan

for pmatch in pmatch_x_remaining['pmatch']:
    pmatch_x_remaining.iloc[pmatch_x_remaining[pmatch_x_remaining.pmatch==pmatch].index,
                            pmatch_x_remaining.columns.get_loc('remaining_rate')] = calculate_remaining_pmatch(final_df, pmatch)
    
pmatch_x_remaining.to_csv(os.path.join(basedir, 'barcode%s.%sdb_pmatch_remaining.tab' % (barcode, db)), sep='\t')

In [28]:
evalue_x_remaining = pd.DataFrame()
evalue_x_remaining['evalue'] = range(0, 414)
evalue_x_remaining['remaining_rate'] = np.nan

for evalue in evalue_x_remaining['evalue']:
    evalue_x_remaining.iloc[evalue_x_remaining[evalue_x_remaining.evalue==evalue].index,
                            evalue_x_remaining.columns.get_loc('remaining_rate')] = calculate_remaining_evalue(final_df, evalue)
    
evalue_x_remaining.to_csv(os.path.join(basedir, 'barcode%s.%sdb_evalue_remaining.tab' % (barcode, db)), sep='\t')